In [1]:
#export
"""
This is for quick modifiers, think of them as changing formats
"""
__all__ = ["apply", "applyMp", "applyS",
           "lstrip", "rstrip", "strip",
           "upper", "lower", "replace", "remove", "toFloat", "toInt",
           "sort", "sortF", "consume", "randomize"]
from typing import Callable, Iterator, Any, Union, List
from k1lib.bioinfo.cli.init import patchDefaultDelim, BaseCli, settings, T
import k1lib.bioinfo.cli as cli, numpy as np, torch
import concurrent.futures as futures
import multiprocessing as mp
from functools import partial
import dill, pickle, k1lib

In [2]:
#export
def executeFunc(f, line, args, kwargs):
    import dill
    f = dill.loads(f)
    line = dill.loads(line)
    args = dill.loads(args)
    kwargs = dill.loads(kwargs)
    return f(line, *args, **kwargs)
class applyMp(BaseCli):
    def __init__(self, f:Callable[[T], T], *args, **kwargs):
        """Like :class:`apply`, but execute ``f(row)`` of each row in
multiple processes. Example::

    # returns [3, 2]
    ["abc", "de"] | applyMp(lambda s: len(s)) | deref()
    # returns [5, 6, 9]
    range(3) | applyMp(lambda x, bias: x**2+bias, bias=5) | deref()
    
    # returns [[1, 2, 3], [1, 2, 3]], demonstrating outside vars work
    someList = [1, 2, 3]
    ["abc", "de"] | applyMp(lambda s: someList) | deref()

Internally, this will continuously spawn new jobs up until 80% of all CPU
cores are utilized. On posix systems, the default multiprocessing start method is
``fork()``. This sort of means that all the variables in memory will be copied
over. This might be expensive (might also not, with copy-on-write), so you might
have to think about that. On windows and macos, the default start method is
``spawn``, meaning each child process is a completely new interpreter, so you have
to pass in all required variables and reimport every dependencies. Read more at
https://docs.python.org/3/library/multiprocessing.html#contexts-and-start-methods

.. note::

    Remember that every :class:`~k1lib.bioinfo.cli.init.BaseCli` is also a
    function, meaning that you can do stuff like::

        # returns [['ab', 'ac']]
        [["ab", "cd", "ac"]] | applyMp(startswith("a") | deref()) | deref()

    Also remember that the return result of ``f`` should not be a generator.
    That's why in the example above, there's a ``deref()`` inside f.

:param args: extra arguments to be passed to the function. ``kwargs`` too"""
        super().__init__(); self.f = f; self.args = args; self.kwargs = kwargs
    def __ror__(self, it:Iterator[T]) -> Iterator[T]:
        super().__ror__(it)
        try: f = dill.dumps(self.f)
        except TypeError as e: print(f"Error while trying to pickle {self.f}."); raise e
        args = dill.dumps(self.args); kwargs = dill.dumps(self.kwargs)
        p = mp.Pool(mp.cpu_count()*4//5)
        fs = [p.apply_async(executeFunc, [f, dill.dumps(line), args, kwargs]) for line in it]
        return (r.get() for r in fs)

In [3]:
assert ["abc", "de"] | applyMp(lambda s: len(s)) | cli.deref() == [3, 2]
assert range(3) | applyMp(lambda x, bias: x**2+bias, bias=5) | cli.deref() == [5, 6, 9]
someList = [1, 2, 3]
assert ["abc", "de"] | applyMp(lambda s: someList) | cli.deref() == [[1, 2, 3], [1, 2, 3]]
assert [["ab", "cd", "ac"]] | applyMp(cli.startswith("a") | cli.deref()) | cli.deref() == [['ab', 'ac']]

In [4]:
#export
class apply(BaseCli):
    def __init__(self, f:Callable[[str], str], column:int=None):
        """Applies a function f to every line

:param column: if not None, then applies the function to that column only"""
        super().__init__(); self.f = f; self.column = column
    def __ror__(self, it:Iterator[str]):
        super().__ror__(it); f = self.f; c = self.column
        if c is None: return (f(line) for line in it)
        else: return ([(e if i != c else f(e)) 
                       for i, e in enumerate(row)] for row in it)
class applyS(BaseCli):
    def __init__(self, f:Callable[[T], T]):
        """Like :class:`apply`, but much simpler, just operating on the entire input
object, essentially"""
        super().__init__(); self.f = f
    def __ror__(self, it:T) -> T:
        super().__ror__(it); return self.f(it)
def lstrip(column:int=None, char:str=None):
    """Strips left of every line.
Example::

    # returns ['12 ', '34']
    ["  12 ", " 34"] | lstrip() | deref()"""
    return apply(lambda e: e.lstrip(char), column)
def rstrip(column:int=None, char:str=None):
    """Strips right of every line"""
    return apply(lambda e: e.rstrip(char), column)
def strip(column:int=None, char:str=None):
    """Strips both sides of every line"""
    return apply(lambda e: e.strip(char), column)

In [5]:
assert ["  12 ", " 34"] | lstrip() | cli.deref() == ['12 ', '34']

In [6]:
#export
def upper(column:int=None):
    """Makes all characters uppercase.
Example::

    # returns ['ABCDE', '123R']
    ["abcde", "123r"] | upper() | deref()"""
    return apply(lambda e: e.upper(), column)
def lower(column:int=None):
    """Makes all characters lowercase"""
    return apply(lambda e: e.lower(), column)

In [7]:
assert ["abcde", "123r"] | upper() | cli.deref() == ['ABCDE', '123R']

In [8]:
#export
def replace(s:str, target:str=None, column:int=None):
    """Replaces substring `s` with `target` for each line.
Example::

    # returns ['104', 'ab0c']
    ["1234", "ab23c"] | replace("23", "0") | deref()

:param target: if not specified, then use the default delimiter specified
    in bioinfo settings"""
    t = patchDefaultDelim(target)
    return apply(lambda e: e.replace(s, t), column)
def remove(s:str, column:int=None):
    """Removes a specific substring in each line."""
    return replace(s, "", column)

In [9]:
assert ["1234", "ab23c"] | replace("23", "0") | cli.deref() == ['104', 'ab0c']

In [10]:
#export
def wrap(f, c): return f if settings["strict"] else cli.isNumeric(c) | f
def toFloat(*columns:List[int]):
    """Converts every row into a float. Excludes non numbers if not in
:ref:`strict mode <bioinfoSettings>`. Example::

    # returns [1, 3, -2.3]
    ["1", "3", "-2.3"] | toFloat() | deref()
    # returns [[1.0, 'a'], [2.3, 'b'], [8.0, 'c']]
    [["1", "a"], ["2.3", "b"], [8, "c"]] | toFloat(0) | deref()

:param columns: if nothing, then will convert each row. If available, then
    convert all the specified columns"""
    if len(columns) > 0:
        return cli.init.serial(*(wrap(apply(lambda e: float(e), c), c) for c in columns))
    else: return wrap(apply(lambda e: float(e), None), None)
def toInt(*columns:List[int]):
    """Converts every row into an integer. Excludes non numbers if not in
:ref:`strict mode <bioinfoSettings>`. Example::

    # returns [1, 3, -2]
    ["1", "3", "-2.3"] | toInt() | deref()

:param columns: if nothing, then will convert each row. If available, then
    convert all the specified columns

See also: :meth:`toFloat`"""
    if len(columns) > 0:
        return cli.init.serial(wrap(apply(lambda e: int(float(e)), c), c) for c in columns)
    else: return wrap(apply(lambda e: int(float(e)), None), None)

In [11]:
assert ["1", "3", "-2.3"] | toFloat() | cli.deref() == [1, 3, -2.3]
assert [["1", "a"], ["2.3", "b"], [8, "c"]] | toFloat(0) | cli.deref() == [[1.0, 'a'], [2.3, 'b'], [8.0, 'c']]
assert ["1", "3", "-2.3"] | toInt() | cli.deref() == [1, 3, -2]

In [12]:
#export
class sort(BaseCli):
    def __init__(self, column:int=0, numeric=True, reverse=False):
        """Sorts all lines based on a specific `column`.
Example::

    # returns [[5, 'a'], [1, 'b']]
    [[1, "b"], [5, "a"]] | ~sort(0) | deref()
    # returns [[2, 3]]
    [[1, "b"], [5, "a"], [2, 3]] | ~sort(1) | deref()
    # errors out, as you can't really compare str with int
    [[1, "b"], [2, 3], [5, "a"]] | sort(1, False) | deref()

:param column: if None, sort rows based on themselves and not an element
:param numeric: whether to convert column to float
:param reverse: False for smaller to bigger, True for bigger to smaller. Use
    :meth:`__invert__` to quickly reverse the order instead of using this param"""
        super().__init__()
        self.column = column; self.reverse = reverse; self.numeric = numeric
        self.filterF = (lambda x: float(x)) if numeric else (lambda x: x)
    def __ror__(self, it:Iterator[str]):
        super().__ror__(it); c = self.column
        if c is None:
            return it | cli.wrapList() | cli.transpose() | sort(0, self.numeric, self.reverse)
        f = self.filterF
        rows = list(it | cli.isNumeric(c) if self.numeric else it)
        def sortF(row):
            if len(row) > c: return f(row[c])
            return float("inf")
        return iter(sorted(rows, key=sortF, reverse=self.reverse))
    def __invert__(self):
        """Creates a clone that has the opposite sort order"""
        return sort(self.column, self.numeric, not self.reverse)

In [13]:
assert [[1, "b"], [5, "a"]] | ~sort(0) | cli.deref() == [[5, 'a'], [1, 'b']]
assert [[1, "b"], [5, "a"], [2, 3]] | ~sort(1) | cli.deref() == [[2, 3]]
try: [[1, "b"], [2, 3], [5, "a"]] | sort(1, False) | cli.deref()
except TypeError: pass

In [14]:
#export
class sortF(BaseCli):
    def __init__(self, f:Callable[[T], float], reverse=False):
        """Sorts rows using a function.
Example::

    # returns ['a', 'aa', 'aaa', 'aaaa', 'aaaaa']
    ["a", "aaa", "aaaaa", "aa", "aaaa"] | sortF(lambda r: len(r)) | deref()
    # returns ['aaaaa', 'aaaa', 'aaa', 'aa', 'a']
    ["a", "aaa", "aaaaa", "aa", "aaaa"] | ~sortF(lambda r: len(r)) | deref()"""
        super().__init__(); self.f = f; self.reverse = reverse
    def __ror__(self, it:Iterator[T]) -> Iterator[T]:
        super().__ror__(it)
        return iter(sorted(list(it), key=self.f, reverse=self.reverse))
    def __invert__(self) -> "sortF":
        return sortF(self.f, not self.reverse)

In [15]:
assert ["a", "aaa", "aaaaa", "aa", "aaaa"] | sortF(lambda r: len(r)) | cli.deref() == ['a', 'aa', 'aaa', 'aaaa', 'aaaaa']
assert ["a", "aaa", "aaaaa", "aa", "aaaa"] | ~sortF(lambda r: len(r)) | cli.deref() == ['aaaaa', 'aaaa', 'aaa', 'aa', 'a']

In [16]:
#export
class consume(BaseCli):
    def __init__(self, f:Union[BaseCli, Callable[[T], None]]):
        r"""Consumes the iterator in a side stream. Returns the iterator.
Kinda like the bash command ``tee``. Example::

    # prints "0\n1\n2" and returns [0, 1, 2]
    range(3) | consume(headOut()) | toList()
    # prints "range(0, 3)" and returns [0, 1, 2]
    range(3) | consume(lambda it: print(it)) | toList()

This is useful whenever you want to mutate something, but don't want to
include the function result into the main stream."""
        super().__init__(); self.f = f
    def __ror__(self, it:T) -> T:
        super().__ror__(it); self.f(it); return it

In [17]:
with k1lib.CaptureStdout() as out:
    assert range(3) | consume(lambda it: print(it)) | cli.toList() == [0, 1, 2]
assert out[0] == "range(0, 3)"
with k1lib.CaptureStdout() as out:
    assert range(3) | consume(cli.headOut()) | cli.toList() == [0, 1, 2]
assert "\n".join(out) == "0\n1\n2"

In [18]:
#export
class randomize(BaseCli):
    def __init__(self, bs=100):
        """Randomize input stream. In order to be efficient, this does not
convert the input iterator to a giant list and yield random values from that.
Instead, this fetches ``bs`` items at a time, randomizes them, returns and
fetch another ``bs`` items. If you want to do the giant list, then just pass
in ``float("inf")``, or ``None``. Example::

    # returns [0, 1, 2, 3, 4], effectively no randomize at all
    range(5) | randomize(1) | deref()
    # returns something like this: [1, 0, 2, 3, 5, 4, 6, 8, 7, 9]. You can clearly see the batches
    range(10) | randomize(3) | deref()
    # returns something like this: [7, 0, 5, 2, 4, 9, 6, 3, 1, 8]
    range(10) | randomize(float("inf")) | deref()
    # same as above
    range(10) | randomize(None) | deref()"""
        super().__init__(); self.bs = bs if bs != None else float("inf")
    def __ror__(self, it:Iterator[T]) -> Iterator[T]:
        super().__ror__(it)
        for batch in it | cli.batched(self.bs, True):
            perms = torch.randperm(len(batch))
            for idx in perms: yield batch[idx]

In [19]:
assert range(10) | randomize(1) | cli.deref() == list(range(10))
a = range(10) | randomize(3) | cli.deref()
assert a[-1] == 9; assert len(a) == 10
assert len(range(10) | randomize(float("inf")) | cli.deref()) == 10
assert len(range(10) | randomize(None) | cli.deref()) == 10

In [23]:
!../../../export.py bioinfo/cli/modifier

Current dir: /home/kelvin/repos/labs/k1lib, ../../../export.py
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 0.1.13
Uninstalling k1lib-0.1.13:
  Successfully uninstalled k1lib-0.1.13
running install
running bdist_egg
running egg_info
creating k1lib.egg-info
writing k1lib.egg-info/PKG-INFO
writing dependency_links to k1lib.egg-info/dependency_links.txt
writing requirements to k1lib.egg-info/requires.txt
writing top-level names to k1lib.egg-info/top_level.txt
writing manifest file 'k1lib.egg-info/SOURCES.txt'
reading manifest file 'k1lib.egg-info/SOURCES.txt'
writing manifest file 'k1lib.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/k1lib
copying k1lib/_learner.py -> build/lib/k1lib
copying k1lib/data.py -> build/lib/k1lib
copying k1lib/selector.py -> build/lib/k1lib
copying k1lib/imports.py -> build/lib/k1lib
copying k1